In [2]:
import pandas as pd
import numpy as np

# 1. โหลดข้อมูลที่ Clean แล้ว
df = pd.read_csv('zeus_dataset_cleaned.csv') # ใช้ไฟล์ล่าสุด
df['datetime'] = pd.to_datetime(df['datetime'])

print(f"ข้อมูลทั้งหมด: {len(df)} แถว")

# --- 2. Feature Engineering (สร้างฟีเจอร์ใหม่) ---
print("กำลังสร้างฟีเจอร์ใหม่...")

# 2.1 ดึงชั่วโมง (Hour) เพื่อจับ Pattern รายวัน (เช้า/สาย/บ่าย/เย็น)
df['hour'] = df['datetime'].dt.hour

# 2.2 สร้างฟีเจอร์ Day/Night (กลางวัน = 1, กลางคืน = 0)
# สมมติ 06:00 - 18:00 คือกลางวัน
df['is_day'] = df['hour'].apply(lambda x: 1 if 6 <= x <= 18 else 0)

# --- 3. Feature Selection (เลือกตัวละครหลัก) ---
# เลือกเฉพาะตัวที่เราวิเคราะห์จาก EDA ว่าเวิร์ค
selected_columns = [
    'datetime', # เก็บไว้ทำ Index
    'temp',     # Target หลัก (The Oracle)
    'humidity', # ช่วยทำนาย Temp & Rain
    'pressure', # ช่วยทำนาย Rain (Zeus's Mood)
    'rain',     # Target รอง
    'uv',       # ช่วยทำนาย Temp (Apollo's Shield)
    'wind_speed',
    'hour',     # Feature ใหม่
    'is_day'    # Feature ใหม่
]

df_final = df[selected_columns].copy()

# --- 4. Data Splitting (แยกข้อมูลสอบ 7 วันสุดท้าย) ---
# ตามใบงานข้อ 6: Deployment
print("กำลังแบ่งข้อมูล Train/Test (ตัด 7 วันสุดท้าย)...")

# หาจุดตัดเวลา (Time Cutoff)
last_date = df_final['datetime'].max()
split_date = last_date - pd.Timedelta(days=7) # ถอยหลังมา 7 วัน

print(f"วันสุดท้ายของข้อมูล: {last_date}")
print(f"จุดตัดแบ่งข้อมูล: {split_date}")

# แบ่งข้อมูล
train_df = df_final[df_final['datetime'] < split_date]
test_df = df_final[df_final['datetime'] >= split_date] # Unseen Data

print(f"จำนวน Train Set (สำหรับสอน AI): {len(train_df)} แถว")
print(f"จำนวน Test Set (สำหรับสอบเก็บคะแนน): {len(test_df)} แถว")

# --- 5. บันทึกไฟล์ ---
train_df.to_csv('zeus_train.csv', index=False)
test_df.to_csv('zeus_test_unseen.csv', index=False)

print("-" * 30)
print("บันทึกไฟล์เสร็จสมบูรณ์:")
print("1. zeus_train.csv  -> เอาไปเข้าโมเดล")
print("2. zeus_test_unseen.csv -> เอาไปทำกราฟโชว์ตอนพรีเซนต์ (ห้ามใช้เทรน)")

ข้อมูลทั้งหมด: 41224 แถว
กำลังสร้างฟีเจอร์ใหม่...
กำลังแบ่งข้อมูล Train/Test (ตัด 7 วันสุดท้าย)...
วันสุดท้ายของข้อมูล: 2026-01-26 15:35:00
จุดตัดแบ่งข้อมูล: 2026-01-19 15:35:00
จำนวน Train Set (สำหรับสอน AI): 39207 แถว
จำนวน Test Set (สำหรับสอบเก็บคะแนน): 2017 แถว
------------------------------
บันทึกไฟล์เสร็จสมบูรณ์:
1. zeus_train.csv  -> เอาไปเข้าโมเดล
2. zeus_test_unseen.csv -> เอาไปทำกราฟโชว์ตอนพรีเซนต์ (ห้ามใช้เทรน)
